In [2]:
%pip install tensorflow -q
%pip install gymnasium -q
%pip install keras -q
%pip install keras-rl2 -q


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Step 1: Create the Environment

This is the OpenAI Gym `Cartpole-v1` environment. The goal is to balance a pole upright and stay within a certain horizontal range for as long as possible. A reward of 1 point is given for each step taken where the pole does not fall or the cart does not go out of bounds. Read more about the environment [here](https://gymnasium.farama.org/environments/classic_control/cart_pole/).

In [10]:
import gymnasium as gym


env = gym.make("CartPole-v1")
state = env.reset()

In [11]:
input_shape = env.observation_space.shape
num_actions = env.action_space.n

print(f"Observation Space: {input_shape}")
print(f"Action Space: {num_actions}")

Observation Space: (4,)
Action Space: 2


In [12]:
print("Initial state: ", state)

Initial state:  (array([-0.00571921,  0.03531359,  0.02441932, -0.04033282], dtype=float32), {})


# Step 2: Build the Model

In [14]:
import setuptools.dist
import tensorflow as tf

# Define the neural network model
def create_model(input_shape, num_actions):
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(shape=input_shape),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dense(num_actions, activation='linear')  # Outputs Q-values for each action, hence two linear nodes
    ])
    return model

In [15]:
model = create_model(input_shape, num_actions)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 24)             │           120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 24)             │           600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │            50 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 770 (3.01 KB)

 Trainable params: 770 (3.01 KB)

 Non-trainable params: 0 (0.00 B)

# Step 3: Train the Model

In [19]:
from collections import deque


learning_rate = 0.0002
gamma = 0.95  # Discount factor for future rewards
epsilon = 1.0  # Initial exploration rate
epsilon_min = 0.1  # Final exploration rate
epsilon_decay = 0.999
batch_size = 32
memory_size = 5000
num_episodes = 500
max_steps_per_episode = 500

replay_memory = deque(maxlen=memory_size)
target_model = create_model(input_shape, num_actions)
target_model.set_weights(model.get_weights())

In [20]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='mse')

#Target model is not compiled because it's weights are not getting updated during training

In [21]:
import numpy as np


# Epsilon-greedy policy for action selection -> Read more here: https://www.geeksforgeeks.org/epsilon-greedy-algorithm-in-reinforcement-learning/
def choose_action(state, epsilon):
    if np.random.rand() < epsilon:
        return np.random.choice(num_actions)  # Explore
    q_values = model.predict(state[np.newaxis], verbose=0)
    return np.argmax(q_values[0])  # Exploit

In [22]:
import random

# Train the model using a batch of experiences
def train_model():
    if len(replay_memory) < batch_size:
        return  # Wait until enough experiences are stored

    batch = random.sample(replay_memory, batch_size)
    states, actions, rewards, next_states, dones = map(np.array, zip(*batch))

    # Compute target Q-values
    next_q_values = target_model.predict(next_states, verbose=0)
    max_next_q_values = np.max(next_q_values, axis=1)
    target_q_values = rewards + gamma * max_next_q_values * (1 - dones)

    # Update the model
    q_values = model.predict(states, verbose=0)
    for i, action in enumerate(actions):
        q_values[i, action] = target_q_values[i]

    model.fit(states, q_values, epochs=1, verbose=0)


In [24]:
max_reward = 0

# Training loop
for episode in range(num_episodes):
    state, _ = env.reset()
    done = False
    total_reward = 0

    for step in range(max_steps_per_episode):
        action = choose_action(state, epsilon)  # Choose action
        next_state, reward, terminated, truncated, _ = env.step(action)
        
        done = terminated or truncated

        # Store the experience
        replay_memory.append((state, action, reward, next_state, done))

        # Train the model
        train_model()

        state = next_state
        total_reward += reward

        if done:
            break

    # Update epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    # Sync weights with the target model periodically
    if episode % 20 == 0:
        target_model.set_weights(model.get_weights())

    if total_reward > max_reward:
        model.save(f"model/dqn-model-max.keras")
        max_reward = total_reward
    
    print(f"Episode {episode + 1}: Total Reward = {total_reward}")


env.close()

Episode 1: Total Reward = 10.0
Episode 2: Total Reward = 42.0
Episode 3: Total Reward = 13.0
Episode 4: Total Reward = 26.0
Episode 5: Total Reward = 13.0
Episode 6: Total Reward = 8.0
Episode 7: Total Reward = 18.0
Episode 8: Total Reward = 15.0
Episode 9: Total Reward = 38.0
Episode 10: Total Reward = 15.0
Episode 11: Total Reward = 30.0
Episode 12: Total Reward = 53.0
Episode 13: Total Reward = 38.0
Episode 14: Total Reward = 14.0
Episode 15: Total Reward = 19.0
Episode 16: Total Reward = 14.0
Episode 17: Total Reward = 23.0
Episode 18: Total Reward = 17.0
Episode 19: Total Reward = 11.0
Episode 20: Total Reward = 36.0
Episode 21: Total Reward = 11.0
Episode 22: Total Reward = 36.0
Episode 23: Total Reward = 28.0
Episode 24: Total Reward = 13.0
Episode 25: Total Reward = 17.0
Episode 26: Total Reward = 28.0
Episode 27: Total Reward = 42.0
Episode 28: Total Reward = 21.0
Episode 29: Total Reward = 11.0
Episode 30: Total Reward = 12.0
Episode 31: Total Reward = 24.0
Episode 32: Total 

In [25]:
model.save("model/dqn-model.keras")

# Model Trained!

Congratulations! You have trained a reinforcement model. To visualize its performance, run the `test.py` file. You can also replace `model/dqn-model.keras` with `model/dqn-model-max.keras` to see how the two models compare